In [209]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model  import LogisticRegression

from sklearn import  ensemble, preprocessing, metrics

In [333]:
dt = pd.read_csv('../train.csv' , encoding='big5')

In [334]:
dt.shape

(100000, 132)

In [335]:
def data_cl(dt):
    dt = dt.drop("CUS_ID" ,axis=1)
    
    ohi = pd.get_dummies(dt["GENDER"])
    dt = dt.drop("GENDER",axis=1)
    dt = pd.concat([ohi,dt],axis=1)
    
    ohi = pd.get_dummies(dt["AGE"])
    dt = dt.drop("AGE",axis=1)
    dt = pd.concat([ohi,dt],axis=1)

    labelencoder = LabelEncoder()
    for i in dt.keys():    
        for a in dt[i]:
            if  a is not None and isinstance(a,str):
                dt[i] = labelencoder.fit_transform(dt[i].astype(str))
                break
    
    for i in dt.keys():
        dt[i] = dt[i].fillna(dt[i].mean())
    
    return dt    
    

In [336]:
dt = data_cl(dt)
dt.shape

(100000, 135)

In [337]:
def logreg(dt):
    x = dt.drop("Y1",axis=1)
    y = dt["Y1"]
    train_X, test_X, train_y, test_y = train_test_split(x, y, test_size = 0.3)

    lr=LogisticRegression()
    lr.fit(train_X,train_y)


    # 印出截距
#     print(lr.intercept_)
    # 印出係數
    a = abs(lr.coef_)[0]
    c =lr.coef_[0]
    a = list(a)
    c = list(c)
    newa = list(c)
    newa.sort()
    x = []
    for i in newa:
        if abs(i) < 0.01:
            x.append(dt.columns[c.index(i)])
    return x

In [359]:
dropx= logreg(dt)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [360]:
dropx

['TERMINATION_RATE',
 'IF_ISSUE_INSD_D_IND',
 'CLC_CUR_NUM',
 'APC_1ST_AGE',
 'L1YR_C_CNT']

In [372]:
pp = []

pp.append(dropx)

In [373]:
pp

[['TERMINATION_RATE',
  'IF_ISSUE_INSD_D_IND',
  'CLC_CUR_NUM',
  'APC_1ST_AGE',
  'L1YR_C_CNT']]

In [361]:
xt = dt.drop("Y1",axis=1)
xt = xt.drop(dropx,axis=1)
xt.shape

(100000, 129)

In [350]:
from keras import models
from keras import layers
from keras.layers.core import Dense , Dropout , Activation

model = models.Sequential()
model.add(layers.Dense(1024 ,input_shape=(xt.shape[1],)))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(32))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(16))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(8))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(4))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(layers.Dense(2))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [342]:
xt.shape

(100000, 127)

In [352]:
# ft = dt.drop("Y1",axis=1)
ft = dt.drop(dropx,axis=1)
fix_dt = ft.sample(frac = 1)

fraud_dt = fix_dt.loc[dt['Y1'] == 1]
non_fraud_dt = fix_dt.loc[dt['Y1'] == 0][:2000]

normal_distributed_dt = pd.concat([fraud_dt, non_fraud_dt])

new_dtf = normal_distributed_dt.sample(frac=1)
# new_dt = normal_distributed_dt.sample(frac=1, random_state=42)

new_dtf.shape

(4000, 126)

In [353]:
x = new_dtf.drop("Y1",axis=1)
y = new_dtf["Y1"]

# x = dt.drop("Y1",axis=1)
# y = dt["Y1"]

model.fit(x,y, epochs=200, batch_size=134 ,verbose=1)

Epoch 1/200
4000/4000 [==============================] - 2s 534us/step - loss: 0.3928 - acc: 0.5255
Epoch 2/200
4000/4000 [==============================] - 1s 302us/step - loss: 0.3313 - acc: 0.5410
Epoch 3/200
4000/4000 [==============================] - 1s 321us/step - loss: 0.3011 - acc: 0.5617
Epoch 4/200
4000/4000 [==============================] - 1s 314us/step - loss: 0.2818 - acc: 0.5788
Epoch 5/200
4000/4000 [==============================] - 1s 349us/step - loss: 0.2769 - acc: 0.5900
Epoch 6/200
4000/4000 [==============================] - 1s 351us/step - loss: 0.2693 - acc: 0.5965
Epoch 7/200
4000/4000 [==============================] - 1s 360us/step - loss: 0.2635 - acc: 0.6033
Epoch 8/200
4000/4000 [==============================] - 1s 334us/step - loss: 0.2558 - acc: 0.6100
Epoch 9/200
4000/4000 [==============================] - 1s 330us/step - loss: 0.2516 - acc: 0.6135
Epoch 10/200
4000/4000 [==============================] - 1s 352us/step - loss: 0.2473 - acc: 0.6277

4000/4000 [==============================] - 2s 395us/step - loss: 0.1807 - acc: 0.7510
Epoch 83/200
4000/4000 [==============================] - 2s 431us/step - loss: 0.1846 - acc: 0.7427
Epoch 84/200
4000/4000 [==============================] - 2s 461us/step - loss: 0.1766 - acc: 0.7588
Epoch 85/200
4000/4000 [==============================] - 2s 417us/step - loss: 0.1719 - acc: 0.7700
Epoch 86/200
4000/4000 [==============================] - 1s 313us/step - loss: 0.1735 - acc: 0.7657
Epoch 87/200
4000/4000 [==============================] - 1s 304us/step - loss: 0.1752 - acc: 0.7638
Epoch 88/200
4000/4000 [==============================] - 1s 311us/step - loss: 0.1775 - acc: 0.7567
Epoch 89/200
4000/4000 [==============================] - 1s 310us/step - loss: 0.1744 - acc: 0.7628
Epoch 90/200
4000/4000 [==============================] - 1s 372us/step - loss: 0.1747 - acc: 0.7623
Epoch 91/200
4000/4000 [==============================] - 2s 502us/step - loss: 0.1739 - acc: 0.7678
Epo

4000/4000 [==============================] - 1s 354us/step - loss: 0.1566 - acc: 0.7990
Epoch 163/200
4000/4000 [==============================] - 2s 535us/step - loss: 0.1568 - acc: 0.7945
Epoch 164/200
4000/4000 [==============================] - 2s 415us/step - loss: 0.1552 - acc: 0.7977
Epoch 165/200
4000/4000 [==============================] - 2s 437us/step - loss: 0.1627 - acc: 0.7847
Epoch 166/200
4000/4000 [==============================] - 1s 319us/step - loss: 0.1553 - acc: 0.7977
Epoch 167/200
4000/4000 [==============================] - 1s 330us/step - loss: 0.1621 - acc: 0.7812
Epoch 168/200
4000/4000 [==============================] - 2s 422us/step - loss: 0.1599 - acc: 0.7915
Epoch 169/200
4000/4000 [==============================] - 1s 307us/step - loss: 0.1620 - acc: 0.7862
Epoch 170/200
4000/4000 [==============================] - 1s 326us/step - loss: 0.1565 - acc: 0.7980
Epoch 171/200
4000/4000 [==============================] - 2s 431us/step - loss: 0.1648 - acc: 0

In [364]:
p = model.evaluate(x,y)  

4000/4000 [==============================] - 1s 175us/step


In [369]:
sss = []

In [370]:
sss.append(p[1])

In [371]:
sss

[0.8952500224113464]

In [377]:
round(p[1]*100,2)

89.53

In [363]:
test_y_predicted = model.predict_classes(xt)
p = model.evaluate(xt,dt["Y1"])

ValueError: Error when checking input: expected dense_231_input to have shape (125,) but got array with shape (129,)

# 預測

In [327]:
df = pd.read_csv('../test.csv' , encoding='big5')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [328]:
df = data_cl(df)
df = df.drop(dropx,axis=1)
df.shape

(150000, 124)

In [329]:
test_y_predicted = model.predict_classes(df)

In [330]:
test_y_predicted.sum()

37867

In [331]:
fin = pd.read_csv('../submit_test.csv' , encoding='big5')
fin["Ypred"] = test_y_predicted
fin.to_csv('submit_test7665.csv')